In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 모델

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, multiply, Cropping1D
from tensorflow.keras.models import Model

def build_mcnn_model(input_shape):
    input_layer = Input(shape=input_shape)  # (80, 4)

    # Encoder1: 단순 CNN
    x1 = Conv1D(filters=50, kernel_size=16, strides=2, padding='valid', activation='tanh')(input_layer)
    x1 = Conv1D(filters=30, kernel_size=8, strides=2, padding='valid', activation='tanh')(x1)
    x1 = MaxPooling1D(pool_size=2, padding='valid')(x1)  # (None, 6, 30)

    # Encoder2: 복잡 CNN
    x2 = Conv1D(filters=50, kernel_size=6, strides=1, padding='valid', activation='tanh')(input_layer)
    x2 = Conv1D(filters=40, kernel_size=6, strides=1, padding='valid', activation='tanh')(x2)
    x2 = MaxPooling1D(pool_size=2, padding='valid')(x2)
    x2 = Conv1D(filters=30, kernel_size=6, strides=1, padding='valid', activation='tanh')(x2)
    x2 = Conv1D(filters=30, kernel_size=6, strides=2, padding='valid', activation='tanh')(x2)
    x2 = MaxPooling1D(pool_size=2, padding='valid')(x2)  # (None, 6, 30)

    # 길이 확인
    print("x1 shape:", x1.shape)
    print("x2 shape:", x2.shape)

    # multiply 병합
    merged = multiply([x1, x2])

    ## LSTM ##


## 데이터 불러오기

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

# CSV 불러오기
csv_path = "/content/drive/MyDrive/KSPHM-data-challenge/data_csv/train1_extracted_features_with_rul.csv"
df = pd.read_csv(csv_path)

# 입력 (X), 라벨 (y) 분리
X = df.drop(columns=["RUL"]).values  # shape: (samples, features)
y = df["RUL"].values  # shape: (samples,)

# 3D 형태로 변환 (MCNN용): (samples, time steps, channels)
# 예: time_steps = 80, channels = 4  → 320개 feature일 경우
X_reshaped = X.reshape(X.shape[0], 80, 4)

# Train/Validation Split
X_train, X_val, y_train, y_val = train_test_split(X_reshaped, y, test_size=0.2, random_state=42)

# y의 shape을 (N, 1)로 변경 (회귀 모델의 출력과 맞춤)
y_train = y_train.reshape(-1, 1)
y_val = y_val.reshape(-1, 1)

In [ ]:
print("X_train shape:", X_train.shape)  # 예상: (N, 80, 4)
print("y_train shape:", y_train.shape)  # 예상: (N, 1)

## 모델 학습

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

model = build_mcnn_model(input_shape=(X_train.shape[1], X_train.shape[2]))
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

early_stop = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(X_train, y_train,
                    epochs=50,
                    batch_size=32,
                    validation_data=(X_val, y_val),
                    callbacks=[early_stop])

## 학습 확인

In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Val Loss')
plt.legend()
plt.title('MCNN train')
plt.xlabel('Epoch')
plt.ylabel('MSE Loss')
plt.show()

## 모델 평가